In [1]:
!pip install datasets==2.11
!pip install datasets transformers[sentencepiece]
!pip install sacrebleu rouge_score evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.15
    Uninstalling multiprocess-0.70.15:
      Successfully uninstalled multiprocess-0.70.15
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This 

In [2]:
from datasets import load_dataset, load_metric
import pandas as pd
import evaluate 
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import DatasetDict
import numpy as np
import evaluate

sacrebleu_metric = load_metric("sacrebleu")
rouge_metric = evaluate.load('rouge')
ter_metric = evaluate.load("ter")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_32/3696305817.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  sacrebleu_metric = load_metric("sacrebleu")


In [3]:
def preprocess_dataset(data):
    conditions = [data.ref_tox < data.trn_tox ]
    values = ['true']
    data['swap'] = np.select(conditions, values)
    
    is_swap = data['swap'] == 'true'
    data.loc[is_swap, ['reference', 'translation', 'ref_tox', 'trn_tox']] = (
        data.loc[is_swap, ['translation', 'reference', 'trn_tox', 'ref_tox']].values
        )
    
    index_drop = data[(data['ref_tox'] <= 0.8) | (data['trn_tox'] >= 0.2) ].index
    data.drop(index_drop, inplace=True)
    data.drop(columns=["swap"], axis=1, inplace=True)
    return data

In [4]:
d = pd.read_table("/kaggle/input/paranmt/filtered.tsv")
d = preprocess_dataset(d)
d.to_csv("converted.csv")

In [5]:
d.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"if Alkar floods her with her mental waste, it ...","If Alkar is flooding her with psychic waste, t...",0.785171,0.010309,0.981983,0.014195
1,1,you're becoming disgusting.,Now you're getting nasty.,0.749687,0.071429,0.999039,0.065473
3,3,"monkey, you have to wake up.","Ah! Monkey, you've got to snap out of it.",0.664333,0.309524,0.994215,0.053362
4,4,I have orders to kill her.,I've got orders to put her down.,0.726639,0.181818,0.999348,0.009402
5,5,I'm not gonna have a child... ...with the same...,I'm not going to breed kids with a genetic dis...,0.703185,0.206522,0.950956,0.035846


In [6]:
raw_datasets = load_dataset("csv", data_files="/kaggle/working/converted.csv")
raw_datasets = raw_datasets['train'].train_test_split(test_size=1-20000/raw_datasets.num_rows["train"], seed=42)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d76025ff401f46ff/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
raw_datasets.num_rows

{'train': 20000, 'test': 469290}

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 469290
    })
})

In [9]:
model_checkpoint = "s-nlp/t5-paranmt-detox"
#model_checkpoint = "s-nlp/bart-base-detox"
#model_checkpoint = "t5-base"

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [11]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate tox to detox: "
else:
    prefix = ""

In [12]:
max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["reference"]]
    targets = [ex for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
preprocess_function(raw_datasets['train'][:2])

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[13959, 12, 226, 12, 16379, 10, 25, 2516, 78, 5375, 189, 63, 16, 851, 13, 722, 5, 1], [13959, 12, 226, 12, 16379, 10, 27, 131, 1114, 12, 214, 3, 99, 8, 3, 17, 7085, 130, 490, 42, 59, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[3, 3626, 5375, 189, 63, 1612, 16, 851, 13, 8, 722, 5, 1], [27, 131, 1114, 12, 214, 3, 99, 3, 3405, 130, 490, 3534, 32, 4849, 42, 59, 5, 1]]}

In [14]:
raw_datasets_train = raw_datasets['train'].train_test_split(test_size=0.3, seed=42)
raw_datasets_test = raw_datasets_train['test'].train_test_split(test_size=0.5, seed=42)


ds_splits = DatasetDict({
    'train': raw_datasets_train['train'],
    'valid': raw_datasets_test['train'],
    'test': raw_datasets_test['test']
})

In [15]:
ds_splits

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 14000
    })
    valid: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 3000
    })
})

In [16]:
tokenized_datasets = ds_splits.map(preprocess_function, batched=True)

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [18]:
source_lang = "toxic"
target_lang = "detoxic"
batch_size = 16

In [19]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    report_to="none"
)

In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [21]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    sacrebleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    ter_result = ter_metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    result = {"bleu": sacrebleu_result["score"],
            "rouge1": rouge_result["rouge1"],
             "rouge2": rouge_result["rouge2"],
             "TER": ter_result["score"]}
    print(result)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [22]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Ter,Gen Len
1,2.045900,1.759455,21.570300,0.549800,0.316300,66.117000,13.161300
2,1.807600,1.725588,22.153800,0.554500,0.321300,65.607000,13.159000
3,1.748700,1.719213,22.164600,0.555300,0.321800,65.547600,13.134700


{'bleu': 21.570334217471157, 'rouge1': 0.5498069745139224, 'rouge2': 0.31630148870653685, 'TER': 66.11703056768559}
{'bleu': 22.153753046310197, 'rouge1': 0.5544721199081696, 'rouge2': 0.3213305380467577, 'TER': 65.60698689956332}
{'bleu': 22.164639011807218, 'rouge1': 0.5552651349317351, 'rouge2': 0.3217650178915302, 'TER': 65.54759825327511}


TrainOutput(global_step=2625, training_loss=1.842638421921503, metrics={'train_runtime': 964.1841, 'train_samples_per_second': 43.56, 'train_steps_per_second': 2.723, 'total_flos': 2293468625387520.0, 'train_loss': 1.842638421921503, 'epoch': 3.0})

In [24]:
trainer.evaluate()

{'bleu': 22.164639011807218, 'rouge1': 0.5552651349317351, 'rouge2': 0.3217650178915302, 'TER': 65.54759825327511}


{'eval_loss': 1.7192133665084839,
 'eval_bleu': 22.1646,
 'eval_rouge1': 0.5553,
 'eval_rouge2': 0.3218,
 'eval_TER': 65.5476,
 'eval_gen_len': 13.1347,
 'eval_runtime': 96.2894,
 'eval_samples_per_second': 31.156,
 'eval_steps_per_second': 1.952,
 'epoch': 3.0}

In [25]:
trainer.save_model()

In [26]:
predictions, labels, metrics = trainer.predict(tokenized_datasets["test"], metric_key_prefix="predict")

{'bleu': 23.683064803364434, 'rouge1': 0.5604341939749399, 'rouge2': 0.33502215287002785, 'TER': 64.4874786667131}


In [27]:
my_predictions = tokenizer.batch_decode(
                    predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )

In [28]:
my_predictions[:15]

['I know, I hate that ethnic group.',
 'now right here, Phil backs up, pushes his foot into the cart, push',
 'This is my snob.',
 'He dismissed the idea that he would dominate the fishermen as a goblin, for even',
 'Chris, what the hell?',
 "Germain (a royal suburb of Paris) and he's busy with the princely",
 '"Here, Valentine, I don\'t know if you deserve a kiss or',
 'They came here to kill him when he was a little boy.',
 "I'll notify the police, but it's useless.",
 "I've already shot the brain.",
 'Get out of here with your boobs!',
 'Oh, my career!',
 'I played you.',
 '"because they asked," growled the big man.',
 'Ostaran sat on the knees of the warrior and slashed his']

In [29]:
ds_splits["test"]["reference"][:15]

['I know, I hate that ethnic group too.',
 'Now right here, Phil backs up, pushes his butt into the cart, pushes me into the cans!',
 "this fucker's mine.",
 'he dismissed the idea that he would dominate the fishermen as goblins, for even the dumbest of them had long defied his will.',
 'Chris, what the hell?!',
 'Germain (a royal suburb of Paris) and busying himself with the princely pastimes of riding, shooting, and screwing high-born Frenchwomen.',
 '"dear Valentine, I don\'t know if you deserve a kiss or a beating for your amber eyes.',
 'they came here to kill him when he was a little boy.',
 "I'll notify the police, but it's useless.",
 "I've already shot the brain.",
 'get out of here with your boobs!',
 'oh, fuck my career!',
 "I played you. You're rubbish.",
 '"Because they goddamn asked for it," growled the big man.',
 "Ostaran dropped to the warrior's knees and put his knife against his throat."]

In [30]:
trainer.state.log_history

[{'loss': 2.0459,
  'learning_rate': 1.6205714285714287e-05,
  'epoch': 0.57,
  'step': 500},
 {'eval_loss': 1.7594554424285889,
  'eval_bleu': 21.5703,
  'eval_rouge1': 0.5498,
  'eval_rouge2': 0.3163,
  'eval_TER': 66.117,
  'eval_gen_len': 13.1613,
  'eval_runtime': 98.2076,
  'eval_samples_per_second': 30.548,
  'eval_steps_per_second': 1.914,
  'epoch': 1.0,
  'step': 875},
 {'loss': 1.8611,
  'learning_rate': 1.2396190476190477e-05,
  'epoch': 1.14,
  'step': 1000},
 {'loss': 1.8076,
  'learning_rate': 8.586666666666667e-06,
  'epoch': 1.71,
  'step': 1500},
 {'eval_loss': 1.7255876064300537,
  'eval_bleu': 22.1538,
  'eval_rouge1': 0.5545,
  'eval_rouge2': 0.3213,
  'eval_TER': 65.607,
  'eval_gen_len': 13.159,
  'eval_runtime': 97.3265,
  'eval_samples_per_second': 30.824,
  'eval_steps_per_second': 1.932,
  'epoch': 2.0,
  'step': 1750},
 {'loss': 1.7705,
  'learning_rate': 4.7771428571428575e-06,
  'epoch': 2.29,
  'step': 2000},
 {'loss': 1.7487,
  'learning_rate': 9.6761904

In [31]:
train_loss = []
eval_loss = []
step = []
eval_step = []
for log in trainer.state.log_history:
    if "loss" in log.keys():
        train_loss.append(log["loss"])
        step.append(log["step"])
    if "eval_loss" in log.keys():
        eval_loss.append(log["eval_loss"])
        eval_step.append(log["step"])

In [ ]:
import matplotlib.pyplot as plt


plt.plot(step, train_loss)
plt.plot(eval_step, eval_loss)
plt.legend(["train_loss", "eval_loss"], loc ="lower right") 
plt.show()

In [33]:
trainer.state.log_history

[{'loss': 2.0459,
  'learning_rate': 1.6205714285714287e-05,
  'epoch': 0.57,
  'step': 500},
 {'eval_loss': 1.7594554424285889,
  'eval_bleu': 21.5703,
  'eval_rouge1': 0.5498,
  'eval_rouge2': 0.3163,
  'eval_TER': 66.117,
  'eval_gen_len': 13.1613,
  'eval_runtime': 98.2076,
  'eval_samples_per_second': 30.548,
  'eval_steps_per_second': 1.914,
  'epoch': 1.0,
  'step': 875},
 {'loss': 1.8611,
  'learning_rate': 1.2396190476190477e-05,
  'epoch': 1.14,
  'step': 1000},
 {'loss': 1.8076,
  'learning_rate': 8.586666666666667e-06,
  'epoch': 1.71,
  'step': 1500},
 {'eval_loss': 1.7255876064300537,
  'eval_bleu': 22.1538,
  'eval_rouge1': 0.5545,
  'eval_rouge2': 0.3213,
  'eval_TER': 65.607,
  'eval_gen_len': 13.159,
  'eval_runtime': 97.3265,
  'eval_samples_per_second': 30.824,
  'eval_steps_per_second': 1.932,
  'epoch': 2.0,
  'step': 1750},
 {'loss': 1.7705,
  'learning_rate': 4.7771428571428575e-06,
  'epoch': 2.29,
  'step': 2000},
 {'loss': 1.7487,
  'learning_rate': 9.6761904

In [ ]:
inputs = tokenizer("You are nigger", return_tensors="pt").input_ids
predictions = model.generate(inputs)
tokenizer.batch_decode(
                    predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )